In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time

In [20]:
df = pd.read_csv('testimony2010-2022.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df

date                                             target
0      3/2/2022  Chair Pro Tempore Powell submitted identical r...
1     1/13/2022  Chairman Brown, Ranking Member Toomey, and oth...
2     1/11/2022  Chairman Brown, Ranking Member Toomey, and oth...
3    11/30/2021  Chair Powell submitted identical remarks to th...
4     9/28/2021  Chair Powell submitted identical remarks to th...
..          ...                                                ...
133    9/2/2010          Chairman Angelides, Vice Chairman Thom...
134    9/1/2010          Chairman Angelides, Vice Chairman Thom...
135   7/21/2010  Chairman Bernanke presented identical remarks ...
136   7/20/2010  Chairman Watt, Ranking Member Paul, and member...
137   7/20/2010          Chairman Bayh, Ranking Member Corker, ...

[138 rows x 2 columns]

In [22]:
nomination_idx = []

for idx in df.index:
    text = df.loc[idx]['target']
    if 'nomination' in text:
        nomination_idx.append(idx)
        
nomination_idx

[1, 28, 62, 136]

In [23]:
df0 = df.copy()
df0.drop(nomination_idx, axis=0, inplace=True)
df0

date                                             target
0      3/2/2022  Chair Pro Tempore Powell submitted identical r...
2     1/11/2022  Chairman Brown, Ranking Member Toomey, and oth...
3    11/30/2021  Chair Powell submitted identical remarks to th...
4     9/28/2021  Chair Powell submitted identical remarks to th...
5     7/14/2021  Chair Powell submitted identical remarks to th...
..          ...                                                ...
132   9/24/2010          Chairman Frank, Ranking Member Bachus,...
133    9/2/2010          Chairman Angelides, Vice Chairman Thom...
134    9/1/2010          Chairman Angelides, Vice Chairman Thom...
135   7/21/2010  Chairman Bernanke presented identical remarks ...
137   7/20/2010          Chairman Bayh, Ranking Member Corker, ...

[134 rows x 2 columns]

In [24]:
len(df), len(df0)

(138, 134)

In [29]:
df0 = df0[['target', 'date']]
df0

target       date
0    Chair Pro Tempore Powell submitted identical r... 2022-03-02
2    Chairman Brown, Ranking Member Toomey, and oth... 2022-01-11
3    Chair Powell submitted identical remarks to th... 2021-11-30
4    Chair Powell submitted identical remarks to th... 2021-09-28
5    Chair Powell submitted identical remarks to th... 2021-07-14
..                                                 ...        ...
132          Chairman Frank, Ranking Member Bachus,... 2010-09-24
133          Chairman Angelides, Vice Chairman Thom... 2010-09-02
134          Chairman Angelides, Vice Chairman Thom... 2010-09-01
135  Chairman Bernanke presented identical remarks ... 2010-07-21
137          Chairman Bayh, Ranking Member Corker, ... 2010-07-20

[134 rows x 2 columns]

In [31]:
df0['date'] = pd.to_datetime(df0['date'])
df0['year'] = df0['date'].dt.year
df0['month'] = df0['date'].dt.month
df0['quarter'] = df0['date'].dt.quarter
df0

target       date  year  \
0    Chair Pro Tempore Powell submitted identical r... 2022-03-02  2022   
2    Chairman Brown, Ranking Member Toomey, and oth... 2022-01-11  2022   
3    Chair Powell submitted identical remarks to th... 2021-11-30  2021   
4    Chair Powell submitted identical remarks to th... 2021-09-28  2021   
5    Chair Powell submitted identical remarks to th... 2021-07-14  2021   
..                                                 ...        ...   ...   
132          Chairman Frank, Ranking Member Bachus,... 2010-09-24  2010   
133          Chairman Angelides, Vice Chairman Thom... 2010-09-02  2010   
134          Chairman Angelides, Vice Chairman Thom... 2010-09-01  2010   
135  Chairman Bernanke presented identical remarks ... 2010-07-21  2010   
137          Chairman Bayh, Ranking Member Corker, ... 2010-07-20  2010   

     month  quarter  
0        3        1  
2        1        1  
3       11        4  
4        9        3  
5        7        3  
..     ...      ...  
132      9        3  
133      9        3  
134      9        3  
135      7        3  
137      7        3  

[134 rows x 5 columns]

In [32]:
df0.to_csv('testimony2010-2022_without_nomination_hearing.csv', index=False)

In [33]:
df1 = pd.read_csv('testimony2010-2022_without_nomination_hearing.csv')
df1

target        date  year  \
0    Chair Pro Tempore Powell submitted identical r...  2022-03-02  2022   
1    Chairman Brown, Ranking Member Toomey, and oth...  2022-01-11  2022   
2    Chair Powell submitted identical remarks to th...  2021-11-30  2021   
3    Chair Powell submitted identical remarks to th...  2021-09-28  2021   
4    Chair Powell submitted identical remarks to th...  2021-07-14  2021   
..                                                 ...         ...   ...   
129          Chairman Frank, Ranking Member Bachus,...  2010-09-24  2010   
130          Chairman Angelides, Vice Chairman Thom...  2010-09-02  2010   
131          Chairman Angelides, Vice Chairman Thom...  2010-09-01  2010   
132  Chairman Bernanke presented identical remarks ...  2010-07-21  2010   
133          Chairman Bayh, Ranking Member Corker, ...  2010-07-20  2010   

     month  quarter  
0        3        1  
1        1        1  
2       11        4  
3        9        3  
4        7        3  
..     ...      ...  
129      9        3  
130      9        3  
131      9        3  
132      7        3  
133      7        3  

[134 rows x 5 columns]

In [34]:
from transformers import pipeline

In [35]:
qa = pipeline('question-answering')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)
All model checkpoint layers were used when initializing TFDistilBertForQuestionAnswering.

All the layers of TFDistilBertForQuestionAnswering were initialized from the model checkpoint at distilbert-base-cased-distilled-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [36]:
for idx in df1.index:
    text = df1.loc[idx]['target']
    answer = qa(question='What are they going to do?', context=text)
    print(answer)

C:\Users\Nayoun Kim\anaconda3\envs\crypto\lib\site-packages\transformers\tokenization_utils_base.py:703: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
C:\Users\Nayoun Kim\anaconda3\envs\crypto\lib\site-packages\transformers\pipelines\question_answering.py:297: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  p_mask = np.asarray(


{'score': 0.5286934971809387, 'start': 3010, 'end': 3034, 'answer': 'promote a long expansion'}
{'score': 0.08809416741132736, 'start': 3834, 'end': 3888, 'answer': 'We will remain vigilant about new and emerging threats'}
{'score': 0.46651118993759155, 'start': 504, 'end': 562, 'answer': 'keeping people from returning to work or looking for a job'}
{'score': 0.055032555013895035, 'start': 6365, 'end': 6436, 'answer': 'Treasury preferred equity of $13.9 billion was returned to the Treasury'}
{'score': 0.2876710891723633, 'start': 1740, 'end': 1762, 'answer': 'search for better ones'}
{'score': 0.21832767128944397, 'start': 9961, 'end': 9976, 'answer': 'asset purchases'}
{'score': 0.14034739136695862, 'start': 9175, 'end': 9209, 'answer': 'examine bank practices accordingly'}
{'score': 0.35917848348617554, 'start': 7023, 'end': 7070, 'answer': 'end issuance of new LIBOR contracts by year-end'}
{'score': 0.48392489552497864, 'start': 1503, 'end': 1572, 'answer': 'provide the economy the 

{'score': 0.5475919842720032, 'start': 10150, 'end': 10189, 'answer': 'finalization of the remaining standards'}
{'score': 0.7497389912605286, 'start': 2918, 'end': 2941, 'answer': 'pursue advanced degrees'}
{'score': 0.1425299346446991, 'start': 5331, 'end': 5369, 'answer': 'reduce their pace of fiscal tightening'}
{'score': 0.055647365748882294, 'start': 9783, 'end': 9836, 'answer': 'move forward with such reforms as quickly as possible'}
{'score': 0.09086667001247406, 'start': 1540, 'end': 1625, 'answer': 'internalize the costs that their failure would impose on the broader financial system'}
{'score': 0.45835819840431213, 'start': 4786, 'end': 4856, 'answer': 'file all the required reports with the appropriate government agencies'}
{'score': 0.2873767912387848, 'start': 201, 'end': 248, 'answer': 'combat money laundering and terrorist financing'}
{'score': 0.9355559945106506, 'start': 11832, 'end': 11867, 'answer': 'reducing the federal budget deficit'}
{'score': 0.3449207842350006

In [37]:
for idx in df1.index:
    text = df1.loc[idx]['target']
    answer = qa(question='What about economic situation?', context=text)
    print(answer)

{'score': 0.12891556322574615, 'start': 438, 'end': 496, 'answer': 'The implications for the U.S. economy are highly uncertain'}
{'score': 0.5796687602996826, 'start': 3849, 'end': 3857, 'answer': 'vigilant'}
{'score': 0.3463057577610016, 'start': 254, 'end': 293, 'answer': 'The economy has continued to strengthen'}
{'score': 0.09482204169034958, 'start': 581, 'end': 620, 'answer': 'the economy has continued to strengthen'}
{'score': 0.2346731573343277, 'start': 579, 'end': 663, 'answer': 'Today I will review the current economic situation before turning to monetary policy'}
{'score': 0.09501897543668747, 'start': 1587, 'end': 1648, 'answer': 'The economic downturn has not fallen equally on all Americans'}
{'score': 0.5981202721595764, 'start': 3030, 'end': 3038, 'answer': 'improves'}
{'score': 0.20749998092651367, 'start': 2476, 'end': 2486, 'answer': 'weaknesses'}
{'score': 0.06689488142728806, 'start': 1620, 'end': 1690, 'answer': 'the path of the economy continues to depend on the 

KeyboardInterrupt: 